In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
import glob
import zipfile

In [41]:
#unzip files
filenames = glob.glob('*zip')
print(filenames)
for file in filenames:
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall("apartments_pl")

['apartments_pl_2024_02.csv.zip', 'apartments_pl_2023_09.csv.zip', 'apartments_pl_2023_10.csv.zip', 'apartments_pl_2023_11.csv.zip', 'apartments_pl_2023_08.csv.zip', 'apartments_pl_2024_01.csv.zip', 'apartments_pl_2023_12.csv.zip']


In [45]:
#load files 
filenames = glob.glob('apartments_pl/*csv')
df = pd.DataFrame()
for file in filenames:
    df_temp = pd.read_csv(file)
    #make date value
    date = pd.to_datetime(file.split('_')[3] + file.split('_')[4].split('.')[0] + '01')
    #add date column and date value for each row #what should the type be?
    df_temp["date"] = date
    df = pd.concat([df, df_temp])
df.shape[0]

117259

In [50]:
#drop observations
df = df.dropna()

#convert to numerical data
df['condition'] = df['condition'].map({'low': 0, 'premium': 1})
df['type'] = df['type'].map({'apartmentBuilding': 1, 'blockOfFlats': 2, 'tenement': 3})
for col in ['hasParkingSpace', 'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom']:
    df[col] = df[col].map({'no': 0, 'yes': 1})

for col in ['hasParkingSpace', 'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom', 'condition', 'type']:
    df[col] = df[col].astype(np.int8)

#drop features
toDrop = ['id', 'ownership'] #buildMaterial, city
df = df.drop(toDrop, axis=1)


In [ ]:
#drop low variance featurse
sel = VarianceThreshold(threshold=.05)
sel.fit(df/df.mean())
mask = sel.get_support() 
#Still need longtude, latitude, and buildYear
df = df.loc[:, mask]

In [46]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 117259 entries, 0 to 16360
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    117259 non-null  object        
 1   city                  117259 non-null  object        
 2   type                  91264 non-null   object        
 3   squareMeters          117259 non-null  float64       
 4   rooms                 117259 non-null  float64       
 5   floor                 96041 non-null   float64       
 6   floorCount            115690 non-null  float64       
 7   buildYear             97426 non-null   float64       
 8   latitude              117259 non-null  float64       
 9   longitude             117259 non-null  float64       
 10  centreDistance        117259 non-null  float64       
 11  poiCount              117259 non-null  float64       
 12  schoolDistance        117133 non-null  float64       
 13  clini

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price,date
count,117259.000000,117259.000000,96041.000000,115690.000000,97426.000000,117259.000000,117259.000000,117259.000000,117259.000000,117133.000000,116763.000000,117088.000000,117121.000000,116936.000000,113878.000000,117070.000000,1.172590e+05,117259
mean,59.367188,2.701848,3.304412,5.235699,1985.299509,52.045362,19.470735,4.318678,20.690037,0.415219,0.973939,0.520115,0.373511,0.351381,1.441477,0.363755,7.569387e+05,2023-10-29 00:58:51.628275968
min,25.000000,1.000000,1.000000,1.000000,1850.000000,49.978999,14.447127,0.020000,0.000000,0.002000,0.001000,0.001000,0.001000,0.001000,0.006000,0.001000,1.500000e+05,2023-08-01 00:00:00
25%,44.650000,2.000000,2.000000,3.000000,1965.000000,51.111693,18.518341,1.960000,7.000000,0.175000,0.355000,0.239000,0.157000,0.115000,0.580000,0.143000,4.990000e+05,2023-09-01 00:00:00
50%,55.400000,3.000000,3.000000,4.000000,1994.000000,52.195920,19.895674,3.920000,13.000000,0.290000,0.675000,0.393000,0.265000,0.231000,1.119000,0.241000,6.790000e+05,2023-11-01 00:00:00
75%,69.700000,3.000000,4.000000,6.000000,2015.000000,52.441367,20.991350,6.120000,24.000000,0.469000,1.237000,0.625000,0.419000,0.412000,2.054000,0.408000,8.990000e+05,2024-01-01 00:00:00
max,150.000000,6.000000,29.000000,29.000000,2024.000000,54.606460,23.207128,16.940000,212.000000,4.946000,4.998000,4.970000,4.961000,4.985000,5.000000,4.992000,3.250000e+06,2024-02-01 00:00:00
std,21.661587,0.923175,2.503613,3.266448,34.244332,1.345636,1.788524,2.860922,24.401407,0.475026,0.897756,0.509243,0.458528,0.478090,1.103144,0.472445,3.904491e+05,NaN


In [47]:
df.to_csv('full_dataset.csv', index=False)

In [51]:
df.columns

Index(['city', 'type', 'squareMeters', 'rooms', 'floor', 'floorCount',
       'buildYear', 'latitude', 'longitude', 'centreDistance', 'poiCount',
       'schoolDistance', 'clinicDistance', 'postOfficeDistance',
       'kindergartenDistance', 'restaurantDistance', 'collegeDistance',
       'pharmacyDistance', 'buildingMaterial', 'condition', 'hasParkingSpace',
       'hasBalcony', 'hasElevator', 'hasSecurity', 'hasStorageRoom', 'price',
       'date'],
      dtype='object')